In [14]:
import random
import datetime
import pandas as pd

# Excepciones personalizadas
class SaldoNegativoError(Exception):
    pass

# Clase Cliente para manejar los datos de cada cliente
class Cliente:
    def __init__(self, nombre):
        self.nombre = nombre
        self.saldo = round(random.uniform(10000, 50000), 2)  # Saldo inicial aleatorio mayor a 10,000
    
    def actualizar_saldo(self, monto, tipo_transaccion):
        if tipo_transaccion in ['retiro', 'transferencia']:
            if self.saldo < monto:
                return False  # No se puede procesar la transacción
            self.saldo -= monto
        elif tipo_transaccion == 'depósito':
            self.saldo += monto
        return True  # Transacción procesada con éxito
    
    def __str__(self):
        return f"{self.nombre}, Saldo: {self.saldo:.2f}"

# Clase Transaccion para manejar las transacciones
class Transaccion:
    def __init__(self, id_transaccion, cliente, monto, tipo_transaccion, fecha, sucursal, destino):
        self.id_transaccion = id_transaccion
        self.cliente = cliente
        self.monto = monto
        self.tipo_transaccion = tipo_transaccion
        self.fecha = fecha
        self.sucursal = sucursal
        self.destino = destino
    
    def procesar(self):
        return self.cliente.actualizar_saldo(self.monto, self.tipo_transaccion)
    
    def to_dict(self):
        # Procesa la transacción para actualizar el saldo
        self.procesar()
        return {
            "ID_transaccion": self.id_transaccion,
            "Nombre cliente": self.cliente.nombre,
            "monto de transacción": self.monto,
            "tipo transaccion": self.tipo_transaccion,
            "fecha transacción": self.fecha.strftime("%b-%y"),
            "saldo actual": self.cliente.saldo,
            "sucursal-origen": self.sucursal,
            "sucursal-destino": self.destino if self.tipo_transaccion == 'transferencia' else '-'
        }

# Clase para generar datos aleatorios
class GeneradorDatos:
    nombres = [
    "Luis", "María", "José", "Ana", "Carlos", "Sofía", "Juan", "Isabella", "Pedro", "Valeria",
    "Jorge", "Gabriela", "Miguel", "Natalia", "Andrés", "Laura", "David", "Paola", "Daniel", "Camila",
    "Alejandro", "Lucía", "Francisco", "Elena", "Santiago", "Carmen", "Roberto", "Daniela", "Mateo", "Juliana",
    "Fernando", "Andrea", "Luisana", "Esteban", "Martina", "Ricardo", "Margarita", "Hugo", "Claudia", "Oscar",
    "Pilar", "Ramón", "Silvia", "Tomás", "Vanessa", "Julio", "Teresa", "Emilia", "Gustavo", "Rosa"
    ]

    apellidos = [
    "García", "Martínez", "Rodríguez", "López", "González", "Pérez", "Hernández", "Sánchez", "Ramírez", "Torres",
    "Flores", "Gómez", "Morales", "Ruiz", "Alvarez", "Mendoza", "Vásquez", "Castro", "Ortega", "Guerrero",
    "Jaramillo", "Bravo", "Cruz", "Reyes", "Vega", "Campos", "Salazar", "Jiménez", "Araya", "Rivas",
    "Navarro", "Marquez", "Córdoba", "Arias", "Escobar", "Bermúdez", "Montoya", "Suárez", "Téllez", "Hurtado",
    "Gallegos", "Roldán", "Pineda", "Serna", "Moya", "Peña", "Acosta", "Serrano", "Beltrán", "Cano"
    ]

    sucursales = ['Quito', 'Guayaquil', 'Cuenca', 'Ibarra', 'Ambato', 'Riobamba']
    tipos_transaccion = ['deposito', 'retiro', 'transferencia']

    @staticmethod
    def generar_nombre():
        return f"{random.choice(GeneradorDatos.nombres)} {random.choice(GeneradorDatos.apellidos)}"

    @staticmethod
    def generar_fecha():
        start_date = datetime.date(2022, 1, 1)
        end_date = datetime.date(2024, 7, 31)
        return start_date + datetime.timedelta(days=random.randint(0, (end_date - start_date).days))
    
    @staticmethod
    def generar_transacciones(n_transacciones, n_clientes):
        clientes = {GeneradorDatos.generar_nombre(): Cliente(GeneradorDatos.generar_nombre()) for _ in range(n_clientes)}
        transacciones = []
        for i in range(1, n_transacciones + 1):
            cliente_nombre = random.choice(list(clientes.keys()))
            cliente = clientes[cliente_nombre]
            monto = round(random.uniform(0.01, 5000), 2)
            tipo_transaccion = random.choice(GeneradorDatos.tipos_transaccion)
            fecha = GeneradorDatos.generar_fecha()
            sucursal = random.choice(GeneradorDatos.sucursales)
            destino = random.choice(GeneradorDatos.sucursales) if tipo_transaccion == 'transferencia' else '-'
            transaccion = Transaccion(i, cliente, monto, tipo_transaccion, fecha, sucursal, destino)
            # Procesar la transacción para actualizar el saldo y agregarla solo si se pudo procesar
            if transaccion.procesar():
                transacciones.append(transaccion)
        return transacciones

# Función para crear y visualizar el dataset
def crear_dataset_df(n_transacciones, n_clientes):
    transacciones = GeneradorDatos.generar_transacciones(n_transacciones, n_clientes)
    dataset = pd.DataFrame([transaccion.to_dict() for transaccion in transacciones])
    return dataset

# Generar dataset para 7000 transacciones y 500 clientes
dataset = crear_dataset_df(7000, 500)
print(dataset.head(10))  # Mostrar las primeras 10 filas del dataset


   ID_transaccion     Nombre cliente  monto de transacción tipo transaccion  \
0               1       Rosa Escobar               2986.05         depósito   
1               2  Claudia Hernández               3666.48           retiro   
2               3  Francisco Morales               1702.75    transferencia   
3               4       José Mendoza                433.29         depósito   
4               5       Tomás Ortega                688.78           retiro   
5               6    Elena Hernández               3970.97         depósito   
6               7       Miguel Araya               1155.38           retiro   
7               8    Isabella Campos               2162.69           retiro   
8               9        Daniel Ruiz               2529.13           retiro   
9              10    Martina Córdoba               2628.43           retiro   

  fecha transacción  saldo actual sucursal-origen sucursal-destino  
0            Sep-22      39323.76          Ambato            

In [15]:
print(dataset)

dataset.shape

      ID_transaccion     Nombre cliente  monto de transacción  \
0                  1       Rosa Escobar               2986.05   
1                  2  Claudia Hernández               3666.48   
2                  3  Francisco Morales               1702.75   
3                  4       José Mendoza                433.29   
4                  5       Tomás Ortega                688.78   
...              ...                ...                   ...   
6787            6996      María Córdoba               2662.85   
6788            6997     Pedro Martínez               1078.42   
6789            6998       Luis Sánchez               1280.21   
6790            6999        Emilia Cruz                  1.56   
6791            7000       María Campos               4971.03   

     tipo transaccion fecha transacción  saldo actual sucursal-origen  \
0            depósito            Sep-22      39323.76          Ambato   
1              retiro            Sep-22       1370.29          Ibarra   


(6792, 8)

In [16]:
# descargar data set 
dataset.to_csv('dataset.csv')